In [ ]:
#Algo asi hay que hacer en la tarea
import cmath
import numpy as np
#Primero que nada necesitamos encontrar el valor de las matrices locales de las lineas, por lo que aquí se colocan los datos de la linea y despues se modelan por el modelo Pi para luego poder sumarlas al Y_buss
Ybuss = np.zeros((6, 6), dtype=complex)
Lista_datos_Lineas= [("1-2",0.031,0.126,0.062),
                     ("1-5",0.031,0.126,0.062),
                     ("2-3",0.063,0.252,0.122),
                     ("3-5",0.053,0.210,0.102),
                     ("3-6",0.084,0.236,0.164)]; # (linea,r,x,b)
for i in Lista_datos_Lineas:
    L=150
    Linea=i[0].split("-")
    Linea_a=int(Linea[0])-1
    Linea_b=int(Linea[1])-1
    z = complex(i[1],i[2])
    y = complex(0,i[3])
    Zc = cmath.sqrt(z/y)
    gama = cmath.sqrt(z*y)
    Z_modelo=Zc*gama*L
    Y_Serie=1/Z_modelo

    Y_Shunt=gama*L/(Zc*2)
    Y_Shunt=Y_Shunt.imag*1j
    
    A=Y_Serie+Y_Shunt
    B=-Y_Serie
    C=B
    D=A

    coordenadas = [(Linea_a, Linea_a), (Linea_a, Linea_b), 
                   (Linea_b, Linea_a), (Linea_b, Linea_b)]
    valores_a_sumar = [A, B, C, D]
    for coord, valor in zip(coordenadas, valores_a_sumar):
        fila, columna = coord
        Ybuss[fila, columna] += valor

############################Trafos######################################

Lista_datos_Trafos= [("4-5",0,0.1,0),
                     ("4-6",0,0.1,0)]; # (linea,r,x,b)
for i in Lista_datos_Trafos:
    Linea=i[0].split("-")
    Linea_a=int(Linea[0])-1
    Linea_b=int(Linea[1])-1

    Z_trafo=complex(i[1],i[2])
    Y_trafo=1/Z_trafo
    A=Y_trafo
    B=-A
    C=B
    D=A

    valores_a_sumar = [A, B, C, D]
    coordenadas = [(Linea_a, Linea_a), (Linea_a, Linea_b), (Linea_b, Linea_a), (Linea_b, Linea_b)]
    for coord, valor in zip(coordenadas, valores_a_sumar):
        fila, columna = coord
        Ybuss[fila, columna] += valor

############################Generación######################################

Lista_datos_Trafos= [("1",0,1.1,0),
                     ("3",0,1.3,0)]; # (linea,r,x,b)
for i in Lista_datos_Trafos:
    Linea=i[0].split("-")
    Linea_a=int(Linea[0])-1
    Z_trafo=complex(i[1],i[2])
    Y_trafo=1/Z_trafo

    A=Y_trafo
    coordenadas = [(Linea_a, Linea_a)]
    valores_a_sumar = [A]
    for coord, valor in zip(coordenadas, valores_a_sumar):
        fila, columna = coord
        Ybuss[fila, columna] += valor
print(Ybuss)